In [ ]:
# import relevant libraries
!pip install numerapi
!pip install ondemand
!pip install py_vollib
!pip install selenium
# !pip install xgboost
import os
import re
import sys
import glob
import json
import time
import math
import shutil
import config
import random
import pickle
import ondemand
import numerapi
import datetime
import py_vollib
import numpy as np
import scipy as sp
import pandas as pd
import requests as rq
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from os import system   
from math import floor
from scipy import stats
from copy import deepcopy
from datetime import datetime
from sklearn import linear_model
from sklearn import neural_network
from sklearn import ensemble
from bs4 import BeautifulSoup as bs
from sklearn import svm
from sklearn.decomposition import PCA
from statsmodels.formula.api import ols
from sklearn import metrics as sklMetrics
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import Lasso, BayesianRidge, LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

## Load Data

In [ ]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

In [ ]:
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

In [ ]:
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

In [ ]:
training_features = training_data[feature_cols]
live_features = tournament_data[feature_cols]

In [ ]:
pca_model = PCA(n_components=100).fit(training_features, training_data.target)
pca_train_x = pca_model.transform(training_features)
pca_test_x = pca_model.transform(live_features)

## Sneetches Model (Multi-Layer Perceptron)

In [ ]:
predictions = neural_network.MLPRegressor(max_iter=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchModID)

## Same As Model (Gradient-Boosted tree)

In [ ]:
predictions = ensemble.GradientBoostingRegressor(n_estimators=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SameAsModID)

## Sneetches Without Model (PCA/Gradient-Boosted Tree)

In [ ]:
predictions = ensemble.GradientBoostingRegressor(n_estimators=500).fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchWOModID)

## Different Than Model (Random Forest)

In [ ]:
predictions = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=-1).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.DifferentThan)

## SneetchSemble (Voting-Regressor)

In [ ]:
predictions_df = tournament_data["id"].to_frame()

In [ ]:
predictions_df['mlpr'] = neural_network.MLPRegressor(max_iter=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df['gbr'] = ensemble.GradientBoostingRegressor(n_estimators=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df['rfr'] = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=-1).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df['svr'] = svm.SVR().fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df['brr'] = BayesianRidge().fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df['lr'] = Lasso().fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df["prediction_kazutsugi"] = (predictions_df['mlpr'] + predictions_df['gbr'] + predictions_df['rfr'] +
                                          predictions_df['svr'] + predictions_df['brr'] + predictions_df['lr'])/6
prediction_df.drop(columns=['mlpr', 'gbr', 'rfr', 'svr', 'brr', 'lr'], inplace=True)

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchSemble)

## SneetchPCA (Voting-Regressor)

In [ ]:
predictions_df = tournament_data["id"].to_frame()

In [ ]:
predictions_df['mlpr'] = neural_network.MLPRegressor(max_iter=500).fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df['gbr'] = ensemble.GradientBoostingRegressor(n_estimators=500).fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df['rfr'] = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=-1).fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df['svr'] = svm.SVR().fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df['brr'] = BayesianRidge().fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df['lr'] = Lasso().fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df["prediction_kazutsugi"] = (predictions_df['mlpr'] + predictions_df['gbr'] + predictions_df['rfr'] +
                                          predictions_df['svr'] + predictions_df['brr'] + predictions_df['lr'])/6
prediction_df.drop(columns=['mlpr', 'gbr', 'rfr', 'svr', 'brr', 'lr'], inplace=True)

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchPCA)

## WIP

In [ ]:
#sorted(sklMetrics.SCORERS.keys())

In [ ]:
#mlpr = GridSearchCV(estimator = neural_network.MLPRegressor(),
#                    param_grid = {'hidden_layer_sizes' : [(i,) for i in range(50, 151, 10)],
#                                  'alpha' : [i*.00001 for i in range(8, 13, 1)],
#                                  'max_iter' : [i for i in range(150, 501, 5)]},
#                    cv = 10,
#                    scoring = 'neg_mean_squared_error',
#                    n_jobs = -1,
#                    verbose = 3)
#mlpr = mlpr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#gbr = GridSearchCV(estimator = ensemble.GradientBoostingRegressor(),
#                   param_grid = {'loss' : ['huber'],
#                                 'learning_rate' : [i*.01 for i in range(9, 12, 1)],
#                                 'n_estimators' : [i for i in range(350, 501, 25)],
#                                 'min_samples_split' : [i for i in range(2, 6, 1)],
#                                 'min_samples_leaf' : [i for i in range(1,4,1)],
#                                 'max_depth' : [i for i in range(2, 11, 1)]},
#                   cv = 10,
#                   scoring = 'neg_mean_absolute_error',
#                   n_jobs = -1,
#                   verbose = 3)
#gbr = gbr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#rfr = GridSearchCV(estimator = ensemble.RandomForestRegressor(),
#                   param_grid = {'n_estimators' : [i for i in range(75, 501, 5)],
#                                 'criterion' : ['mse', 'mae'],
#                                 'max_depth' : [i for i in range(2, 11, 1)],
#                                 'min_samples_split' : [i for i in range(2, 11, 1)],
#                                 'min_samples_leaf' : [i for i in range(1,4,1)]},
#                   cv = 10,
#                   scoring = 'neg_mean_squared_error',
#                   n_jobs = -1,
#                   verbose = 3)
#rfr = rfr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#svr = GridSearchCV(estimator = svm.SVR(),
#                   param_grid = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
#                                 'degree' : [i for i in range(2, 6, 1)],
#                                 'gamma' : ['scale', 'auto'],
#                                 'epsilon' : [i*.01 for i in range(5, 16, 1)],
#                                 'cache_size' : 500},
#                   cv = 10,
#                   scoring = 'neg_mean_squared_error',
#                   n_jobs = -1,
#                   verbose = 3)
#svr = svr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#brr = GridSearchCV(estimator = BayesianRidge(),
#                   param_grid = {'n_iter' : [i for i in range(250, 501, 5)],
#                                 'fit_intercept' : [True, False],
#                                 'normalize' : [True, False]},
#                   cv = 10,
#                   scoring = 'neg_mean_squared_error',
#                   n_jobs = -1,
#                   verbose = 3)
#brr = brr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#lr = GridSearchCV(estimator = Lasso(),
#                  param_grid = {'alpha' : [i*.1 for i in range(5, 16, 1)],
#                                'fit_intercept' : [True, False],
#                                'normalize' : [True, False],
#                                'max_iter' : [i for i in range(750, 1251, 5)]},
#                  cv = 10,
#                  scoring = 'neg_mean_squared_error',
#                  n_jobs = -1,
#                  verbose = 3)
#lr = lr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#vr = GridSearchCV(estimator = ensemble.VotingRegressor(ee),
#                  param_grid = {'estimators' : [[('mlpr', mlpr),
#                                                 ('gbr', gbr),
#                                                 ('rfr', rfr),
#                                                 ('svr', svr),
#                                                 ('brr', brr),
#                                                 ('lr', lr)]],
#                                'weights' : [[.25, .15, .15, .15, .15, .15],
#                                             [.15, .25, .15, .15, .15, .15],
#                                             [.15, .15, .25, .15, .15, .15],
#                                             [.15, .15, .15, .25, .15, .15],
#                                             [.15, .15, .15, .15, .25, .15],
#                                             [.15, .15, .15, .15, .15, .25]]},
#                  cv = 10,
#                  scoring = 'neg_mean_squared_error',
#                  n_jobs = -1,
#                  verbose = 3)
#vr = vr.fit(pca_train_x, training_data.target).best_estimator_

In [ ]:
#mlpr = neural_network.MLPRegressor()
#gbr = ensemble.GradientBoostingRegressor()
#rfr = ensemble.RandomForestRegressor()
#svr = svm.SVR()
#brr = BayesianRidge()
#lr = Lasso()
#models = [('mlpr', mlpr),('gbr', gbr),('rfr', rfr),('svr', svr),('brr', brr),('lr', lr)]
#vr = ensemble.VotingRegressor(models, n_jobs=-1)

In [ ]:
#er = vr.fit(pca_train_x, training_data.target)

In [ ]:
#predictions = er.predict(pca_test_x)

In [ ]:
#predictions_df = tournament_data["id"].to_frame()
#predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
#napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
#predictions_df.to_csv("predictions.csv", index=False)
#submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchSemble)